# Segmentasi Citra *Cable Icing* Menggunakan U-Net

## Persiapan model dan fungsi yang akan digunakan

In [ ]:
from tensorflow import keras
from keras import Input, Model, Sequential
from keras.layers import BatchNormalization, Concatenate, Conv2D, Conv2DTranspose, Dropout, MaxPooling2D, RandomFlip, RandomTranslation, RandomZoom, ReLU
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import MeanIoU
from keras.models import load_model
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
import tensorflow as tf
import time
import json
import os

In [ ]:
# Base directory
base = '/content/drive/MyDrive/Tugas IPCV Image Segmentation'

In [ ]:
image_sizes = np.array([1024, 768])
new_image_sizes = np.uint16(image_sizes/4)

rng = np.random.default_rng(seed = 10)

### Pembuatan model u-net

In [ ]:
def doubleConvolution(features, input):

  conv1 = Conv2D(features, (3,3), kernel_initializer = 'he_normal', padding = 'same') (input)
  batch_norm1 = BatchNormalization() (conv1)
  relu1 = ReLU() (batch_norm1)

  conv2 = Conv2D(features, (3,3), kernel_initializer = 'he_normal', padding = 'same') (relu1)
  batch_norm2 = BatchNormalization() (conv2)
  relu2 = ReLU() (batch_norm2)
  return relu2

def contractingPath(features, input):
  convolution = doubleConvolution(features, input)
  pooling = MaxPooling2D((2,2), strides = 2) (convolution)
  dropout = Dropout(0.3) (pooling)
  return convolution, dropout

def expandingPath(features, input, contracting_conv):
  deconvolution = Conv2DTranspose(features, (2,2), kernel_initializer = 'he_normal', strides = 2, padding = 'same') (input)
  concat = Concatenate() ([contracting_conv, deconvolution])
  dropout = Dropout(0.3) (concat)
  convolution = doubleConvolution(features, dropout)
  return convolution

In [ ]:
input = Input(shape=(new_image_sizes[1],new_image_sizes[0],3))

left_conv1, contract1 = contractingPath(64, input)
left_conv2, contract2 = contractingPath(128, contract1)
left_conv3, contract3 = contractingPath(256, contract2)
left_conv4, contract4 = contractingPath(512, contract3)

left_conv5 = doubleConvolution(1024, contract4)
contract5 = Dropout(0.3) (left_conv5)

expand1 = expandingPath(512, contract5, left_conv4)
expand2 = expandingPath(256, expand1, left_conv3)
expand3 = expandingPath(128, expand2, left_conv2)
expand4 = expandingPath(64, expand3, left_conv1)

output = Conv2D(3, (1,1), kernel_initializer = 'he_normal', padding = 'same') (expand4)

u_net_model = Model(input, output)
u_net_model.compile(optimizer='SGD', loss=SparseCategoricalCrossentropy(from_logits = True))

### Fungsi untuk peningkatan citra

In [ ]:
def convertBGR2HSI(imgs):
  blue = imgs[:,:,:,0]
  green = imgs[:,:,:,1]
  red = imgs[:,:,:,2]

  denominator = np.square(red-green)+((red-blue)*(green-blue))
  modified_denominator = np.where(denominator == 0, 10**-10, denominator)

  hue = np.where(denominator > 0, np.arccos(np.clip(0.5*((red-green)+(red-blue))/np.sqrt(modified_denominator), -1, 1)), 0)
  hue[blue>green] = 2*np.pi - hue[blue>green]
  intensity = (red+green+blue)/3
  modified_intensity = np.where(intensity == 0, 10**-10, intensity)
  saturation = np.where(intensity == 0, 0, 1 - np.minimum(np.minimum(red, green),blue)/modified_intensity)
  return np.stack([np.degrees(hue), saturation, intensity], axis = -1)

def convertHSI2BGR(imgs):
  hue = imgs[:,:,:,0]
  saturation = imgs[:,:,:,1]
  intensity = imgs[:,:,:,2]

  hue[hue == 0] = 360
  hue = np.radians(hue)

  blue = np.zeros_like(hue)
  green = np.zeros_like(hue)
  red = np.zeros_like(hue)

  blue = blue + np.where((hue>0)&(hue<=2/3*np.pi),intensity*(1-saturation),0)
  red = red + np.where((hue>0)&(hue<=2/3*np.pi),intensity*(1+(saturation*np.cos(hue))/np.cos(np.pi/3-hue)),0)
  green = green + np.where((hue>0)&(hue<=2/3*np.pi),3*intensity-(red+blue),0)

  red = red + np.where((hue>2/3*np.pi)&(hue<=4/3*np.pi),intensity*(1-saturation),0)
  green = green + np.where((hue>2/3*np.pi)&(hue<=4/3*np.pi),intensity*(1+(saturation*np.cos(hue-2/3*np.pi))/np.cos(np.pi-hue)),0)
  blue = blue + np.where((hue>2/3*np.pi)&(hue<=4/3*np.pi),3*intensity-(red+green),0)

  green = green + np.where((hue>4/3*np.pi)&(hue<=2*np.pi),intensity*(1-saturation),0)
  blue = blue + np.where((hue>4/3*np.pi)&(hue<=2*np.pi),intensity*(1+(saturation*np.cos(hue-4/3*np.pi))/np.cos(5/3*np.pi-hue)),0)
  red = red + np.where((hue>4/3*np.pi)&(hue<=2*np.pi),3*intensity-(green+blue),0)

  return np.clip(np.stack([blue,green,red], axis = -1), 0, 1)

def singleScaleRetinex(img, sigma):
    img = np.float64(img) + 1
    retinex = np.log10(img) - np.log10(cv.GaussianBlur(img, (0, 0), sigma))
    return retinex

def multiScaleRetinex(img, sigma_list, sigma_weight):
    img = np.float64(img)
    retinex = np.zeros_like(img)
    for index, sigma in enumerate(sigma_list):
        retinex += sigma_weight[index] * singleScaleRetinex(img, sigma)
    return (retinex-np.min(retinex))/(np.max(retinex)-np.min(retinex))

def brightenImagesHE(imgs):
  hsi_imgs = convertBGR2HSI(imgs)
  intensities = []
  for img in hsi_imgs:
    intensities.append(cv.equalizeHist((img[:,:,2]*255).astype(np.uint8))/255)
  hsi_imgs[:,:,:,2] = np.array(intensities)
  bgr_imgs = convertHSI2BGR(hsi_imgs)
  return bgr_imgs

def brightenImagesMSR(imgs):
  sigma_list = [15, 80, 250]
  sigma_weight = [1/3, 1/3, 1/3]
  brightened_imgs = list()
  for img in imgs:
    brightened_imgs.append(multiScaleRetinex(img, sigma_list, sigma_weight))
  return np.array(brightened_imgs)

## Persiapan data berisi citra dan *mask*

In [ ]:
augmentation = Sequential(
    [RandomFlip('horizontal'),
     RandomTranslation(0.2, 0.2),
     RandomZoom((-0.2,0.2))]
)

def createTrainingAndTestSet(combined_imgs, split_amount, augment_amount):
  combined_imgs = np.array(combined_imgs)
  rng.shuffle(combined_imgs)
  combined_train_imgs, combined_test_imgs = [combined_imgs[:np.round(split_amount*combined_imgs.shape[0]).astype(np.uint8)], combined_imgs[np.round(split_amount*combined_imgs.shape[0]).astype(np.uint8):]]

  combined_train_imgs = np.repeat(combined_train_imgs, augment_amount, axis = 0)
  augmented_combinations = augmentation(tf.convert_to_tensor(combined_train_imgs), training = True).numpy()
  augmented_train_imgs = augmented_combinations[:,:,:,:3]
  augmented_train_segment_masks = augmented_combinations[:,:,:,3]

  test_imgs = combined_test_imgs[:,:,:,:3]
  test_segment_masks = combined_test_imgs[:,:,:,3]

  return augmented_train_imgs, augmented_train_segment_masks, test_imgs, test_segment_masks

In [ ]:
combined_img_and_masks_without_normal = []
combined_img_and_masks_with_normal = []

image_types = [0,0]

with os.scandir(base + '/IcingCableLineSegmentation/Segmentations') as it:
  for segmentation in it:
    current_image_type = 0
    json_file = open(segmentation)
    converted_file = json.load(json_file)
    img_file = cv.resize(cv.imread(base + '/IcingCableLineSegmentation/Images/' + converted_file['imagePath']).astype(np.float32)/255,(new_image_sizes[0], new_image_sizes[1]))
    segment_mask = np.zeros((768,1024), dtype = np.uint8)
    for mask_shape in converted_file['shapes']:
      if mask_shape['label'] == 'line':
        current_image_type = 1
        cv.fillPoly(segment_mask, [np.int32(np.round(np.array(mask_shape['points']),0))], [1])
      elif mask_shape['label'] == 'line_icing':
        cv.fillPoly(segment_mask, [np.int32(np.round(np.array(mask_shape['points']),0))], [2])
    segment_mask = cv.resize(segment_mask,(new_image_sizes[0], new_image_sizes[1]))
    combined_img = np.append(img_file, segment_mask[:,:,None], axis = 2)
    if current_image_type == 0:
      combined_img_and_masks_without_normal.append(combined_img)
    else:
      combined_img_and_masks_with_normal.append(combined_img)
    image_types[current_image_type] += 1

train_imgs_without_normal, train_segment_masks_without_normal, test_imgs_without_normal, test_segment_masks_without_normal = createTrainingAndTestSet(combined_img_and_masks_without_normal, 0.8, 10)
train_imgs_with_normal, train_segment_masks_with_normal, test_imgs_with_normal, test_segment_masks_with_normal = createTrainingAndTestSet(combined_img_and_masks_with_normal, 0.8, 21)

train_imgs = np.append(train_imgs_without_normal, train_imgs_with_normal, axis = 0)
train_segment_masks = np.append(train_segment_masks_without_normal, train_segment_masks_with_normal, axis = 0)
test_imgs = np.append(test_imgs_without_normal, test_imgs_with_normal, axis = 0)
test_segment_masks = np.append(test_segment_masks_without_normal, test_segment_masks_with_normal, axis = 0)

In [ ]:
print('Images without normal cable:', image_types[0])
print('Images with normal cable:', image_types[1])
print('Total images:', image_types[0] + image_types[1])

### Peningkatan citra *test set*

In [ ]:
he_test_imgs = brightenImagesHE(test_imgs)
msr_test_imgs = brightenImagesMSR(test_imgs)

## Pelatihan model u-net

In [ ]:
u_net = u_net_model
u_net.fit(train_imgs, train_segment_masks, batch_size = 16)
u_net.save(base + '/u_net_model_sparse')

## Hasil prediksi menggunakan model yang sudah dilatih

In [ ]:
def displayImages(img, true, input_list, pred_list, img_names):
  img = (img*255).astype(np.uint8)
  colored_preds = list()
  for pred in pred_list:
    colored_pred = np.full((pred.shape[0],pred.shape[1],3), np.array([0,0,0]), dtype = np.uint8)
    colored_pred[pred==1] = np.array([255,0,0])
    colored_pred[pred==2] = np.array([0,0,255])
    colored_preds.append(colored_pred)
  colored_true = np.full((true.shape[0],true.shape[1],3), np.array([0,0,0]), dtype = np.uint8)
  colored_true[true==1] = np.array([255,0,0])
  colored_true[true==2] = np.array([0,0,255])

  cv.imwrite(base+'/Normal Image.png', img)
  cv.imwrite(base+'/True Mask.png', cv.cvtColor(colored_true, cv.COLOR_RGB2BGR))
  for index in range(len(input_list)):
    cv.imwrite(base+'/Input Image {}.png'.format(img_names[index]), np.round(input_list[index]*255).astype(np.uint8))
    cv.imwrite(base+'/Predicted Mask {}.png'.format(img_names[index]), cv.cvtColor(colored_preds[index], cv.COLOR_RGB2BGR))
    fig = plt.figure(index+1)
    fig.suptitle('Result for {}:'.format(img_names[index]), fontsize = 16)
    fig.subplots_adjust(top = 1.3)
    ax = plt.subplot(141)
    plt.imshow(img)
    ax.set_title('Original Image')
    ax = plt.subplot(142)
    plt.imshow(colored_true)
    ax.set_title('Ground Truth')
    ax = plt.subplot(143)
    plt.imshow(input_list[index])
    ax.set_title('Input Image')
    ax = plt.subplot(144)
    plt.imshow(colored_preds[index])
    ax.set_title('Mask Prediction')

def calculateMeanIoU(true, pred):
  mean_iou = MeanIoU(num_classes = 3)
  mean_iou.update_state(true, pred)
  return mean_iou.result().numpy()

In [ ]:
u_net = load_model(base + '/u_net_model_sparse')

In [ ]:
u_net_predict_normal = np.argmax(u_net.predict(np.array(test_imgs)),axis = -1)
u_net_predict_he = np.argmax(u_net.predict(np.array(he_test_imgs)),axis = -1)
u_net_predict_msr = np.argmax(u_net.predict(np.array(msr_test_imgs)),axis = -1)

### Tampilan dari hasil peningkatan citra dan prediksi *mask* segmentasi

In [ ]:
image_index = 4

all_inputs = [test_imgs[image_index], he_test_imgs[image_index], msr_test_imgs[image_index]]
all_predictions = [u_net_predict_normal[image_index], u_net_predict_he[image_index], u_net_predict_msr[image_index]]
prediction_names = ['Normal Test Set', 'HE Brightened Test Set', 'MSR Brightened Test Set']

displayImages(test_imgs[image_index], test_segment_masks[image_index], all_inputs, all_predictions, prediction_names)

### Kalkulasi rata-rata IoU setiap prediksi dari citra masukan

In [ ]:
print('Normal test set MeanIoU: ' + str(calculateMeanIoU(test_segment_masks, u_net_predict_normal)))
print('HE brightened test set MeanIoU: ' + str(calculateMeanIoU(test_segment_masks, u_net_predict_he)))
print('MSR brightened test set MeanIoU: ' + str(calculateMeanIoU(test_segment_masks, u_net_predict_msr)))

Normal test set MeanIoU: 0.58008224
HE brightened test set MeanIoU: 0.27235022
MSR brightened test set MeanIoU: 0.54899627
